In [42]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
import time

# 需要什么读取什么不要一次性全都读取了，影响程序速度

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

time1 = time.time()

with open('database_ByZCG\\DCBS_DataFrame.pkl', 'rb') as f:  
    DCBS_DataFrame = pickle.load(f)
    

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)

    
# 只提取A区中存活的转债的时间序列信息
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()

# 提取B区信息
B_DF = DTBS_DataFrame['B']
# 计算所有转债每天的涨跌幅，并找出涨幅大于前N个交易日的q分位数涨跌幅的均值的转债

In [43]:
# DataFrame新增列用于存放前一天收盘价、涨跌幅
A_DF.sort_values(by = ['bond_code','date'], inplace=True, ascending=[True,True])
A_DF['lastday_dp'] = A_DF.groupby('bond_code')['dp'].shift(1)
A_DF['change'] = A_DF['dp'] / A_DF['lastday_dp'] - 1
A_DF = pd.merge(A_DF, B_DF[['bond_code', 'sn']].drop_duplicates(), on='bond_code', how='left')

# 设定选择区间为20天，得到低价中点与低价程度两列，利用滑动平均实现
A_DF.sort_values(['bond_code', 'date'], ascending=[True,True], inplace=True)
grouped = A_DF.groupby('bond_code')
n = 10

# 两种不同的计算方式，一种是计算前20天收盘价的最高价、最低价（不含当天），一种是计算前20天收盘价的最高价、最低价（含当天）
# A_DF['highest_price_n_days'] = grouped['dp'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].max(), raw=True).reset_index(level=0, drop=True)
# A_DF['lowest_price_n_days'] = grouped['dp'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].min(), raw=True).reset_index(level=0, drop=True)

A_DF['highest_price_n_days'] = grouped['dp'].rolling(window=n, min_periods=n).max().reset_index(level=0, drop=True)
A_DF['lowest_price_n_days'] = grouped['dp'].rolling(window=n, min_periods=n).min().reset_index(level=0, drop=True)


A_DF['low_price_level'] = A_DF['dp'] / A_DF['lowest_price_n_days'] - 1 
A_DF['low_price_midpoint'] = (A_DF['highest_price_n_days'] + A_DF['lowest_price_n_days']) / 2

A_DF['dp_median'] = A_DF.groupby('date')['dp'].transform('median')
print(A_DF.describe())

                 cpr             dp             bl            trt  \
count  389957.000000  389957.000000  389958.000000  389957.000000   
mean       37.026270     129.798368      17.072551      31.672238   
std        41.619414      61.923378      45.468461     172.796754   
min       -34.222800      18.002000       0.000000       0.000000   
25%        12.807600     108.582000       3.370200       0.736255   
50%        25.939000     117.800000       6.499400       1.932700   
75%        46.813600     130.000000      13.212200       7.488700   
max      1006.244800    3420.400000     500.000000   26250.223530   

                 ytm             yl             dl        ia             qs  \
count  388958.000000  389694.000000  389958.000000  389958.0  389958.000000   
mean       -2.309896       3.547039     166.837546       1.0       0.133163   
std         8.381267       1.777318      75.567464       0.0       0.339751   
min      -426.190000       0.000000     -13.418000       1.0  

In [6]:
"""
# 临时新建的功能,非必要，2023-08-14
q_change = 0.99

end_day = '2023-08-18'
start_day = '2023-07-01'

# 提取B区信息
D_DF = DTBS_DataFrame['D']
day_list = list(D_DF.loc[(D_DF['date'] >= start_day) & (D_DF['date'] <= end_day),'date'])
print(day_list)

print('前', round((1-q_change)*100,2), '%涨跌幅分位点：')

A_DF['quantile_change'] = A_DF.groupby('date')['change'].transform(lambda x: x.dropna().quantile(q_change))

change_quantile_list = []
for day in day_list:
    change_quantile = float(A_DF[(A_DF['date']==day)&(A_DF['bond_code']=='118016.SH')]['quantile_change'])
    change_quantile_list.append(change_quantile)
    print(day, '  ',round(change_quantile*100,2),'%')
print('均值：',round(sum(change_quantile_list)/len(change_quantile_list)*100,2),'%')
"""

['2023-07-03', '2023-07-04', '2023-07-05', '2023-07-06', '2023-07-07', '2023-07-10', '2023-07-11', '2023-07-12', '2023-07-13', '2023-07-14', '2023-07-17', '2023-07-18', '2023-07-19', '2023-07-20', '2023-07-21', '2023-07-24', '2023-07-25', '2023-07-26', '2023-07-27', '2023-07-28', '2023-07-31', '2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04', '2023-08-07', '2023-08-08', '2023-08-09', '2023-08-10', '2023-08-11', '2023-08-14', '2023-08-15', '2023-08-16', '2023-08-17', '2023-08-18']
前 1.0 %涨跌幅分位点：
2023-07-03    7.72 %
2023-07-04    5.16 %
2023-07-05    3.78 %
2023-07-06    4.21 %
2023-07-07    2.21 %
2023-07-10    4.39 %
2023-07-11    5.94 %
2023-07-12    3.05 %
2023-07-13    3.58 %
2023-07-14    3.11 %
2023-07-17    2.86 %
2023-07-18    5.21 %
2023-07-19    3.57 %
2023-07-20    2.25 %
2023-07-21    3.46 %
2023-07-24    4.61 %
2023-07-25    5.69 %
2023-07-26    2.51 %
2023-07-27    1.86 %
2023-07-28    4.63 %
2023-07-31    6.31 %
2023-08-01    3.66 %
2023-08-02    3.78 %
2023-08-03 

In [17]:
# ----------------------------------------------------------------------------------------------------------------------
# --------------------------------------------------- 分页，便于阅读 ---------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------------

In [44]:
day_list  = sorted(list(set(A_DF['date'])))
print(day_list)

"""
设置分位点，时间参数（这些参数可以后期调整）；
q_change（判定债券是否热门的分位点），q_dl（判定债券双低是否较低的分位点）
n(用前n天作为一个周期看债券之前是否热门),m（一个周期内需要几天涨幅较大超过分位点才能认为这只债券之前是热门债券）
"""
time1 = time.time()

q_change = 0.95
n = 10
q_dl = 0.3
q_low_price_level = 0.3
m = 1 

# 使用数据从'2023-04-27'-'2023-07-13',实际测试时间从'2023-5-18'-'2023-07-13',因为需要测试日的前10个交易日的数据去观察这个债券之前是否热门
start_index = day_list.index('2020-01-02')
end_index = day_list.index('2023-08-18')

day_list = day_list[start_index:end_index+1]
test_day_list = day_list[n+1:]


history_hot_codes_Dict = {}


# 获取到目前为止已退市可转债代码，这里的目的是希望在买入时规避掉这些即将退市的转债
delisted_bond_codes = list(B_DF.loc[B_DF['dld'] < '2046-01-01', 'bond_code'])
A_DF['是否属于已退市转债的最后N个交易日（1代表是）'] = 0

A_DF_filtered = A_DF[A_DF['bond_code'].isin(delisted_bond_codes)].copy()

# 然后，按"A"列进行分组，并在"B"列中找出每个组的topN的值
N = 20
top_n = A_DF_filtered.groupby('bond_code')['date'].apply(lambda x: x.sort_values(ascending=False).head(N))

# 然后，你可以根据筛选结果，给"C"列赋值
for index in top_n.index:
    A_DF_filtered.loc[index[1], '是否属于已退市转债的最后N个交易日（1代表是）'] = 1

# 将修改过的部分合并到原始的df中
A_DF.update(A_DF_filtered)
print(A_DF)



A_DF['quantile_change'] = A_DF.groupby('date')['change'].transform(lambda x: x.dropna().quantile(q_change))
A_DF['quantile_dl'] = A_DF.groupby('date')['dl'].transform(lambda x: x.dropna().quantile(q_dl))
A_DF['quantile_low_price_level'] = A_DF.groupby('date')['low_price_level'].transform(lambda x: x.dropna().quantile(q_low_price_level))
print(A_DF)

A_DF['当日是否属于热门转债'] = (A_DF['change'] >= A_DF['quantile_change']).astype(int)
A_DF['当日是否双低较小'] = (A_DF['dl'] <= A_DF['quantile_dl']).astype(int)
A_DF['当日是否低价程度较小'] = (A_DF['low_price_level'] <= A_DF['quantile_low_price_level']).astype(int)

A_DF.sort_values(['bond_code', 'date'], inplace=True)
grouped = A_DF.groupby('bond_code')
A_DF['过去一段时间是热门转债的天数'] = grouped['当日是否属于热门转债'].rolling(window=n+1, min_periods=n+1).apply(lambda x: x[:-1].sum(), raw=True).reset_index(level=0, drop=True)
A_DF['是否属于过去一段时间内的热门转债'] = (A_DF['过去一段时间是热门转债的天数'] >= m).astype(int)
# print(A_DF)

['2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06', '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12', '2017-01-13', '2017-01-16', '2017-01-17', '2017-01-18', '2017-01-19', '2017-01-20', '2017-01-23', '2017-01-24', '2017-01-25', '2017-01-26', '2017-02-03', '2017-02-06', '2017-02-07', '2017-02-08', '2017-02-09', '2017-02-10', '2017-02-13', '2017-02-14', '2017-02-15', '2017-02-16', '2017-02-17', '2017-02-20', '2017-02-21', '2017-02-22', '2017-02-23', '2017-02-24', '2017-02-27', '2017-02-28', '2017-03-01', '2017-03-02', '2017-03-03', '2017-03-06', '2017-03-07', '2017-03-08', '2017-03-09', '2017-03-10', '2017-03-13', '2017-03-14', '2017-03-15', '2017-03-16', '2017-03-17', '2017-03-20', '2017-03-21', '2017-03-22', '2017-03-23', '2017-03-24', '2017-03-27', '2017-03-28', '2017-03-29', '2017-03-30', '2017-03-31', '2017-04-05', '2017-04-06', '2017-04-07', '2017-04-10', '2017-04-11', '2017-04-12', '2017-04-13', '2017-04-14', '2017-04-17', '2017-04-18', '2017-04-19', '2017-04-20', '2017

        bond_code        date      cpr       dp        bl         trt     ytm  \
0       110030.SH  2017-01-03  41.5266  114.040  9.780730    0.373183 -1.6776   
1       110030.SH  2017-01-04  40.3232  114.230  9.780730    0.122792 -1.7348   
2       110030.SH  2017-01-05  40.2584  114.950  9.780730    1.022316 -1.9463   
3       110030.SH  2017-01-06  41.3362  114.860  9.780730    0.493010 -1.9219   
4       110030.SH  2017-01-09  42.6442  115.530  9.780730    0.341181 -2.1219   
...           ...         ...      ...      ...       ...         ...     ...   
389953  128145.SZ  2023-02-20  -0.7001  131.670  1.187342  214.411349 -3.3080   
389954  128145.SZ  2023-02-21  -0.3236  130.449  1.099123   82.549997 -3.0847   
389955  128145.SZ  2023-02-22  -0.5313  127.030  0.976149   59.889218 -2.4404   
389956  128145.SZ  2023-02-23  -0.2378  125.205  0.773509   68.496036 -2.0877   
389957  128145.SZ  2023-02-24  -0.3382  122.499  0.156706  488.807066 -1.5511   

              yl        dl 

In [19]:
# ----------------------------------------------------------------------------------------------------------------------
# 上面这些代码的核心功能是生成    ->    热门可转债+双低（全量数据DTBS，用于回测）.xlsx
# ----------------------------------------------------------------------------------------------------------------------

# 添加若干限定条件
empty_df_select = A_DF[(A_DF['是否属于过去一段时间内的热门转债']==1) & 
                       (A_DF['当日是否双低较小']==1) & 
                       (A_DF['date'] >= test_day_list[0]) &
                       (A_DF['date'] <= test_day_list[-1])].copy()
# print(empty_df_select)

# 最后需要展示的列
output_columns = ['date' ,'bond_code' ,'dp','sn','change', 'dl','cpr','bl','yl','trt', 'ytm','qs','qs30']

empty_df_select = empty_df_select[empty_df_select['是否属于已退市转债的最后N个交易日（1代表是）'] == 0][output_columns].copy()


"""保存到EXCEL得时候重命名列名，便于阅读"""
empty_df_select.rename(columns={'date' : '日期',
                                'bond_code' : '可转债代码',
                                'dp' : '价格（收盘价全价）',
                                'cpr' : '转股溢价率',
                                'dl' : '双低',
                                'change':'涨跌幅',
                                'sn' : '正股名称',
                                'bl' : '债券余额（亿）',
                                'yl' : '剩余期限（年）',
                                'trt' : '换手率',
                                'ytm' : '到期收益率'},inplace=True)

empty_df_select.sort_values(by=['日期','可转债代码'],ascending=[True,True] ,inplace=True)

time2 = time.time()

empty_df_select.to_excel('pre_data_for_backtest\\热门可转债+双低（全量数据DTBS，预处理）.xlsx',index=False)

time3 = time.time()
print(f"运行时间：{time2-time1:.2f}秒, 存储时间：{time3-time2:.2f}秒, 总时间：{time3-time1:.2f}秒")

# ----------------------------------------------------------------------------------------------------------------------
# 上面这些代码的核心功能是生成    ->    热门可转债+双低（全量数据DTBS，用于回测）.xlsx
# ----------------------------------------------------------------------------------------------------------------------

运行时间：6.62秒, 存储时间：5.63秒, 总时间：12.25秒


In [20]:
# ----------------------------------------------------------------------------------------------------------------------
# 下面这些代码的核心功能是生成    ->    热门可转债+低价程度（全量数据DTBS，用于回测）.xlsx
# ----------------------------------------------------------------------------------------------------------------------

# 添加若干限定条件
empty_df_select = A_DF[(A_DF['是否属于过去一段时间内的热门转债']==1) & 
                       (A_DF['当日是否低价程度较小']==1) & 
                       # (A_DF['low_price_level'] <= 0.008) &
                       (A_DF['当日是否双低较小']==1) &
                       (A_DF['dp'] >= 110) &
                       (A_DF['dp'] <= 130) &
                       (A_DF['cpr'] <= 40) &
                       (A_DF['date'] >= test_day_list[0]) &
                       (A_DF['date'] <= test_day_list[-1]) & 
                       (~A_DF['sn'].str.contains("ST"))].copy()
# print(empty_df_select)

# 最后需要展示的列
output_columns = ['date' ,'bond_code' ,'dp','sn','change', 'dl','cpr','bl','yl','trt', 'ytm','qs','qs30','low_price_level']

empty_df_select = empty_df_select[empty_df_select['是否属于已退市转债的最后N个交易日（1代表是）'] == 0][output_columns].copy()


"""保存到EXCEL得时候重命名列名，便于阅读"""
empty_df_select.rename(columns={'date' : '日期',
                                'bond_code' : '可转债代码',
                                'dp' : '价格（收盘价全价）',
                                'cpr' : '转股溢价率',
                                'dl' : '双低',
                                'change':'涨跌幅',
                                'sn' : '正股名称',
                                'bl' : '债券余额（亿）',
                                'yl' : '剩余期限（年）',
                                'trt' : '换手率',
                                'ytm' : '到期收益率'},inplace=True)

empty_df_select.sort_values(by=['日期','可转债代码'],ascending=[True,True] ,inplace=True)

time2 = time.time()

empty_df_select.to_excel('pre_data_for_backtest\\热门可转债+低价程度（全量数据DTBS，用于回测）.xlsx',index=False)

time3 = time.time()
print(f"存储时间：{time3-time2:.2f}秒")

# ----------------------------------------------------------------------------------------------------------------------
# 上面这些代码的核心功能是生成    ->    热门可转债+低价程度（全量数据DTBS，用于回测）.xlsx
# ----------------------------------------------------------------------------------------------------------------------

存储时间：0.66秒


In [45]:
# 2023-08-21 新建

# 添加若干限定条件
empty_df_select = A_DF[(A_DF['是否属于过去一段时间内的热门转债']==1) & 
                       (A_DF['dp'] >= 105) &
                       (A_DF['bl'] <= 5) &
                       (A_DF['ytm'] >= 0.7) &
                       (A_DF['ytm'] < 2) &
                       (A_DF['trt'] >= 2) &
                       (A_DF['trt'] <= 10) &
                       (A_DF['cpr'] <= 50) &
                       # 根据需要确定是否去除ST债
                       (~A_DF['sn'].str.contains("ST")) &
                       (A_DF['date'] >= test_day_list[0]) &
                       (A_DF['date'] <= test_day_list[-1])].copy()
# print(empty_df_select)

# 最后需要展示的列
output_columns = ['date' ,'bond_code' ,'dp','sn','change', 'dl','cpr','bl','yl','trt', 'ytm','qs','qs30']

empty_df_select = empty_df_select[empty_df_select['是否属于已退市转债的最后N个交易日（1代表是）'] == 0][output_columns].copy()


"""保存到EXCEL得时候重命名列名，便于阅读"""
empty_df_select.rename(columns={'date' : '日期',
                                'bond_code' : '可转债代码',
                                'dp' : '价格（收盘价全价）',
                                'cpr' : '转股溢价率',
                                'dl' : '双低',
                                'change':'涨跌幅',
                                'sn' : '正股名称',
                                'bl' : '债券余额（亿）',
                                'yl' : '剩余期限（年）',
                                'trt' : '换手率',
                                'ytm' : '到期收益率'},inplace=True)

empty_df_select.sort_values(by=['日期','可转债代码'],ascending=[True,True] ,inplace=True)

time2 = time.time()

print(empty_df_select)
empty_df_select.to_excel('pre_data_for_backtest\\热门可转债+ytm（全量数据DTBS，用于回测）.xlsx',index=False)


time3 = time.time()
print(f"运行时间：{time2-time1:.2f}秒, 存储时间：{time3-time2:.2f}秒, 总时间：{time3-time1:.2f}秒")


                日期      可转债代码  价格（收盘价全价）  正股名称       涨跌幅        双低    转股溢价率  \
324652  2020-01-17  128042.SZ    110.099  凯中精密  0.006629  133.5807  23.4817   
324653  2020-01-20  128042.SZ    111.000  凯中精密  0.008184  136.0311  25.0311   
189336  2020-01-22  123033.SZ    110.000  金力永磁 -0.012390  131.1765  21.1765   
342362  2020-01-22  128069.SZ    112.599  华森制药 -0.023807  123.8444  11.2454   
342363  2020-01-23  128069.SZ    111.230  华森制药 -0.012158  124.9765  13.7465   
...            ...        ...        ...   ...       ...       ...      ...   
216533  2023-08-09  123082.SZ    115.664  北陆药业 -0.004159  156.6740  41.0100   
250173  2023-08-10  123171.SZ    116.646  共同药业  0.000412  155.9260  39.2800   
216535  2023-08-11  123082.SZ    115.700  北陆药业  0.001732  161.6700  45.9700   
216538  2023-08-16  123082.SZ    115.229  北陆药业 -0.002631  162.7490  47.5200   
216539  2023-08-17  123082.SZ    114.592  北陆药业 -0.005528  162.8320  48.2400   

         债券余额（亿）   剩余期限（年）       换手率   到期收益率   qs  